In [1]:
from tensorflow.keras.layers import (Conv2D, UpSampling2D, Conv2DTranspose, concatenate, MaxPooling2D, 
                                     Activation, Dropout, Cropping2D, Flatten, Dense, BatchNormalization)
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [2]:
IMG_HEIGHT = 300
IMG_WIDTH = 300
TRAIN_IMAGE_PATH = "jpeg/train/"
TEST_IMAGE_PATH = "jpeg/test/"
BATCH_SIZE = 8
AUTO = tf.data.experimental.AUTOTUNE
REG = 0.0005
EPOCHS=40
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.binary_crossentropy,
]

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
train_df["image_name"] = train_df["image_name"].apply(lambda x: TRAIN_IMAGE_PATH + x + ".jpg")
test_df["image_name"] = test_df["image_name"].apply(lambda x: TEST_IMAGE_PATH + x + ".jpg")

In [5]:
train_df_target_1 = train_df[train_df["target"] == 1]
train_df_target_0 = train_df[train_df["target"] == 0]

In [6]:
del train_df

In [7]:
random_target_0 = np.random.randint(low=1, high=train_df_target_0.shape[0], 
                                    size=1 * train_df_target_1.shape[0])

In [8]:
train_df_d = pd.concat([train_df_target_0.iloc[random_target_0], train_df_target_1])

In [9]:
def decode_image(filename, label=None, image_size=(IMG_WIDTH, IMG_HEIGHT)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    #image = tf.cast(image, tf.float32)/255.0
    image = tf.image.resize(image, size=image_size)
    
    if label is None:
        return image
    else:
        return image, label
    
def data_augment(image, label=None):
    image = tf.image.rgb_to_hsv(image)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.adjust_brightness(image, 0.2)
    image = tf.image.rot90(image)
    image = tf.image.central_crop(image, central_fraction=0.3)
    if label is None:
        return image
    else:
        return image, label

In [10]:
# lb = LabelEncoder()
# image_names = train_df["image_name"].values
# train_df["image_name"] = lb.fit_transform(train_df["image_name"].values)
# train_df["target"] = train_df["target"].astype("int")
# train_df.head()
# map_name_no = dict(zip(train_df["image_name"], image_names))
# y_train = train_df["target"]
# x_train = train_df[["image_name"]]


# over = SMOTE(random_state=45, sampling_strategy=0.1)
# under = RandomUnderSampler(sampling_strategy=0.5)
# steps = [('o', over), ('u', under)]
# ppl = Pipeline(steps=steps)
# x_train, y_train = ppl.fit_resample(x_train, y_train)

In [11]:
x_train = train_df_d[["image_name"]]
y_train = train_df_d["target"].astype(np.float32).values
x_train.reset_index(drop=True, inplace=True)

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=45)
# x_train["image_name"] = x_train["image_name"].apply(lambda x: map_name_no[x])
# x_val["image_name"] = x_val["image_name"].apply(lambda x: map_name_no[x])

In [13]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((934, 1), (234, 1), (934,), (234,))

In [14]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [15]:
class_weights

array([0.96887967, 1.03318584])

In [16]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((x_train["image_name"].values, y_train))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .repeat()
                 .shuffle(512)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )

In [17]:
val_dataset = (tf.data.Dataset
               .from_tensor_slices((x_val["image_name"].values, y_val))
               .map(decode_image, num_parallel_calls=AUTO)
               .repeat()
               .shuffle(512)
               .batch(BATCH_SIZE)
               .prefetch(AUTO))

In [18]:
test_dataset = (tf.data.Dataset.from_tensor_slices((test_df.image_name))
                .map(decode_image, num_parallel_calls=AUTO)
                .cache()
                .batch(BATCH_SIZE))

In [19]:
lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=3,
    min_lr=0.000001,
    factor=0.5,
    verbose=1
)

In [20]:
es = EarlyStopping(monitor="val_loss", patience=10)

In [21]:
model_chkpt = ModelCheckpoint(filepath="best_model.h5")

In [22]:

def get_crop_shape(target, refer):
        # width, the 3rd dimension
        print("target: {} {}, refer: {} {}".format(target, target.get_shape(), refer, refer.get_shape()))
        cw = (target.get_shape()[2] - refer.get_shape()[2])
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1])
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

In [23]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)
    return 1 - (numerator + 1) / (denominator + 1)

In [24]:
def model_unet(input_layer, expansion_filters=64, expansion_kernel=(3,3), expansion_pool_size=(2,2),
          contract_filters=64, contract_kernel=(3,3), contract_pool_size=(2,2)):
    
    #64
    lvl_1 = Conv2D(filters=expansion_filters, kernel_size=expansion_kernel, activation="relu", padding="same")(input_layer)
    lvl_1 = Conv2D(filters=expansion_filters, kernel_size=expansion_kernel, activation="relu", padding="same")(lvl_1)
    mp_lvl_1 = MaxPooling2D(expansion_pool_size)(lvl_1)
    mp_lvl_1 = Dropout(0.25)(mp_lvl_1)
    
    #128
    lvl_2 = Conv2D(filters=expansion_filters*2, kernel_size=expansion_kernel, activation="relu", padding="same")(mp_lvl_1)
    lvl_2 = Conv2D(filters=expansion_filters*2, kernel_size=expansion_kernel, activation="relu", padding="same")(lvl_2)
    mp_lvl_2 = MaxPooling2D(expansion_pool_size)(lvl_2)
    mp_lvl_2 = Dropout(0.25)(mp_lvl_2)
    
    #256
    lvl_3 = Conv2D(filters=expansion_filters*3, kernel_size=expansion_kernel, activation="relu", padding="same")(mp_lvl_2)
    lvl_3 = Conv2D(filters=expansion_filters*3, kernel_size=expansion_kernel, activation="relu", padding="same")(lvl_3)
    mp_lvl_3 = MaxPooling2D(expansion_pool_size)(lvl_3)
    mp_lvl_3 = Dropout(0.25)(mp_lvl_3)
    
    #512
    lvl_4 = Conv2D(filters=expansion_filters*4, kernel_size=expansion_kernel, activation="relu", padding="same")(mp_lvl_3)
    lvl_4 = Conv2D(filters=expansion_filters*4, kernel_size=expansion_kernel, activation="relu", padding="same")(lvl_4)
    mp_lvl_4 = MaxPooling2D(expansion_pool_size)(lvl_4)
    mp_lvl_4 = Dropout(0.25)(mp_lvl_4)
    
    #1024
    lvl_5 = Conv2D(filters=expansion_filters*5, kernel_size=expansion_kernel, activation="relu", padding="same")(mp_lvl_4)
    lvl_5 = Conv2D(filters=expansion_filters*5, kernel_size=expansion_kernel, activation="relu", padding="same")(lvl_5)
    
    #d_lvl_4 = Conv2DTranspose(filters=contract_filters*4, kernel_size=contract_kernel, activation="relu", padding="same")(lvl_5)
    d_lvl_4 = UpSampling2D(size=contract_pool_size, data_format="channels_last")(lvl_5)
    ch, cw = get_crop_shape(lvl_4, d_lvl_4)
    ccon_4 = Cropping2D(cropping=(ch, cw), data_format="channels_last")(lvl_4)
    ucon_4 = concatenate([d_lvl_4, ccon_4])
    ucon_4 = Dropout(0.25)(ucon_4)
    ucon_4 = Conv2D(filters=contract_filters*4, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_4)
    ucon_4 = Conv2D(filters=contract_filters*4, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_4)
    
    #d_lvl_3 = Conv2DTranspose(filters=contract_filters*3, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_4)
    d_lvl_3 = UpSampling2D(size=contract_pool_size, data_format="channels_last")(ucon_4)
    ch, cw = get_crop_shape(lvl_3, d_lvl_3)
    ccon_3 = Cropping2D(cropping=(ch, cw), data_format="channels_last")(lvl_3)
    ucon_3 = concatenate([d_lvl_3, ccon_3])
    ucon_3 = Dropout(0.25)(ucon_3)
    ucon_3 = Conv2D(filters=contract_filters*3, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_3)
    ucon_3 = Conv2D(filters=contract_filters*3, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_3)
    
    #d_lvl_2 = Conv2DTranspose(filters=contract_filters*2, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_3)
    d_lvl_2 = UpSampling2D(size=contract_pool_size, data_format="channels_last")(ucon_3)
    ch, cw = get_crop_shape(lvl_2, d_lvl_2)
    ccon_2 = Cropping2D(cropping=(ch, cw), data_format="channels_last")(lvl_2)
    ucon_2 = concatenate([d_lvl_2, ccon_2])
    ucon_2 = Dropout(0.25)(ucon_2)
    ucon_2 = Conv2D(filters=contract_filters*3, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_2)
    ucon_2 = Conv2D(filters=contract_filters*3, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_2)
    
    #d_lvl_1 = Conv2DTranspose(filters=contract_filters*1, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_2)
    d_lvl_1 = UpSampling2D(size=contract_pool_size, data_format="channels_last")(ucon_2)
    ch, cw = get_crop_shape(lvl_1, d_lvl_1)
    ccon_1 = Cropping2D(cropping=(ch, cw), data_format="channels_last")(lvl_1)
    ucon_1 = concatenate([d_lvl_1, ccon_1])
    ucon_1 = Dropout(0.25)(ucon_1)
    ucon_1 = Conv2D(filters=contract_filters*1, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_1)
    ucon_1 = Conv2D(filters=contract_filters*1, kernel_size=contract_kernel, activation="relu", padding="same")(ucon_1)
    
    output = Conv2D(filters=1, kernel_size=(1,1), activation="relu", padding="same")(ucon_1)
    
    flatten = Flatten()(output)
    dense4 = Dense(256, activation='relu')(flatten)
    bn4 = BatchNormalization()(dense4)
    dense3 = Dense(256, activation='relu')(bn4)
    bn3 = BatchNormalization()(dense3)
    dense2 = Dense(128, activation='relu')(bn3)
    bn2 = BatchNormalization()(dense2)
    dense1 = Dense(1, activation="sigmoid")(bn2)
    model = Model(inputs=input_layer, outputs=dense1)
    return model
    
    
     

input_layer = Input((IMG_HEIGHT, IMG_WIDTH, 3))
model = model_unet(input_layer, expansion_filters=16)
model.summary()

target: Tensor("conv2d_7/Identity:0", shape=(None, 37, 37, 64), dtype=float32) (None, 37, 37, 64), refer: Tensor("up_sampling2d/Identity:0", shape=(None, 36, 36, 80), dtype=float32) (None, 36, 36, 80)
target: Tensor("conv2d_5/Identity:0", shape=(None, 75, 75, 48), dtype=float32) (None, 75, 75, 48), refer: Tensor("up_sampling2d_1/Identity:0", shape=(None, 72, 72, 256), dtype=float32) (None, 72, 72, 256)
target: Tensor("conv2d_3/Identity:0", shape=(None, 150, 150, 32), dtype=float32) (None, 150, 150, 32), refer: Tensor("up_sampling2d_2/Identity:0", shape=(None, 144, 144, 192), dtype=float32) (None, 144, 144, 192)
target: Tensor("conv2d_1/Identity:0", shape=(None, 300, 300, 16), dtype=float32) (None, 300, 300, 16), refer: Tensor("up_sampling2d_3/Identity:0", shape=(None, 288, 288, 192), dtype=float32) (None, 288, 288, 192)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         P

In [25]:
model

In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=dice_loss, metrics=METRICS)

In [27]:
history = model.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, es, model_chkpt],
                        steps_per_epoch=x_train.shape[0]//BATCH_SIZE, validation_data=val_dataset,
                        validation_steps=x_val.shape[0]//BATCH_SIZE,
                        class_weight = class_weights)

Train for 116 steps, validate for 29 steps
Epoch 1/40
116/116 [==============================] - 147s 1s/step - loss: 0.2366 - tp: 279.0000 - fp: 225.0000 - tn: 255.0000 - fn: 169.0000 - accuracy: 0.5754 - precision: 0.5536 - recall: 0.6228 - auc: 0.5823 - binary_crossentropy: 1.0628 - val_loss: 0.2553 - val_tp: 45.0000 - val_fp: 18.0000 - val_tn: 79.0000 - val_fn: 90.0000 - val_accuracy: 0.5345 - val_precision: 0.7143 - val_recall: 0.3333 - val_auc: 0.6629 - val_binary_crossentropy: 0.6909
Epoch 2/40
116/116 [==============================] - 105s 909ms/step - loss: 0.2123 - tp: 283.0000 - fp: 207.0000 - tn: 259.0000 - fn: 179.0000 - accuracy: 0.5841 - precision: 0.5776 - recall: 0.6126 - auc: 0.6048 - binary_crossentropy: 2.0798 - val_loss: 0.2534 - val_tp: 112.0000 - val_fp: 54.0000 - val_tn: 47.0000 - val_fn: 19.0000 - val_accuracy: 0.6853 - val_precision: 0.6747 - val_recall: 0.8550 - val_auc: 0.6927 - val_binary_crossentropy: 0.6753
Epoch 3/40
116/116 [===========================

116/116 [==============================] - 122s 1s/step - loss: 0.1640 - tp: 338.0000 - fp: 195.0000 - tn: 291.0000 - fn: 104.0000 - accuracy: 0.6778 - precision: 0.6341 - recall: 0.7647 - auc: 0.6947 - binary_crossentropy: 2.5015 - val_loss: 0.2155 - val_tp: 132.0000 - val_fp: 100.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5690 - val_precision: 0.5690 - val_recall: 1.0000 - val_auc: 0.5000 - val_binary_crossentropy: 3.6353
Epoch 18/40
116/116 [==============================] - 123s 1s/step - loss: 0.1689 - tp: 350.0000 - fp: 209.0000 - tn: 266.0000 - fn: 103.0000 - accuracy: 0.6638 - precision: 0.6261 - recall: 0.7726 - auc: 0.6796 - binary_crossentropy: 2.4850 - val_loss: 0.2435 - val_tp: 119.0000 - val_fp: 113.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.5129 - val_precision: 0.5129 - val_recall: 1.0000 - val_auc: 0.5000 - val_binary_crossentropy: 4.1243
Epoch 19/40
115/116 [============================>.] - ETA: 0s - loss: 0.1575 - tp: 339.

In [28]:
history

In [29]:
pred = model.predict(test_dataset)

In [30]:
pred

array([[0.99926645],
       [0.9994809 ],
       [0.9988951 ],
       ...,
       [0.9998274 ],
       [0.999406  ],
       [0.999828  ]], dtype=float32)

In [31]:
sam = pd.read_csv("sample_submission.csv")

In [32]:
sam.head()


,image_name,target
0,ISIC_0052060,0
1,ISIC_0052349,0
2,ISIC_0058510,0
3,ISIC_0073313,0
4,ISIC_0073502,0


In [33]:
sam["target"] = pred

In [34]:
sam

,image_name,target
0,ISIC_0052060,0.999266
1,ISIC_0052349,0.999481
2,ISIC_0058510,0.998895
3,ISIC_0073313,0.999193
4,ISIC_0073502,0.999834
...,...,...
10977,ISIC_9992485,0.999819
10978,ISIC_9996992,0.999822
10979,ISIC_9997917,0.999827
10980,ISIC_9998234,0.999406


In [35]:
sam.to_csv("dice_loss_unet_2d_images_hsv_augment_equal_weights.csv", index=False)

In [36]:
history

In [37]:
history.__dict__

{'validation_data': None,
 'model': <tensorflow.python.keras.engine.training.Model at 0x1f87a5fba48>,
 '_chief_worker_only': None,
 'params': {'batch_size': None,
  'epochs': 40,
  'steps': 116,
  'samples': 116,
  'verbose': 0,
  'do_validation': True,
  'metrics': ['loss',
   'tp',
   'fp',
   'tn',
   'fn',
   'accuracy',
   'precision',
   'recall',
   'auc',
   'binary_crossentropy',
   'val_loss',
   'val_tp',
   'val_fp',
   'val_tn',
   'val_fn',
   'val_accuracy',
   'val_precision',
   'val_recall',
   'val_auc',
   'val_binary_crossentropy']},
 'epoch': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19],
 'history': {'loss': [0.23659261470210963,
   0.2122536227107048,
   0.1909219577009308,
   0.17756300387454443,
   0.17215824660299153,
   0.17054572084854389,
   0.16909694080722742,
   0.18390476748604198,
   0.16724883720021824,
   0.1626689574713337,
   0.18799499589307556,
   0.17313331995030928,
   0.1633538971

In [38]:
import matplotlib.pyplot as plt

In [39]:
loss_ = history.history["loss"]
val_loss_ = history.history["val_loss"]
epochs = [i for i in range(11)]